In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_C_057' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_C_057
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.004411184114791146
test_rmse: 0.004392649897285058
test_r2: 0.656202133733572
feature number: 3
train_rmse: 0.004383196449609636
test_rmse: 0.004411265060407387
test_r2: 0.6527816007667263
feature number: 4
train_rmse: 0.004249866568787606
test_rmse: 0.004320124167651256
test_r2: 0.6669035064424486
feature number: 5
train_rmse: 0.004121235725067467
test_rmse: 0.00426022146478554
test_r2: 0.6757584074062853
feature number: 6
train_rmse: 0.0041188631319995315
test_rmse: 0.004276541040699773
test_r2: 0.6733387606953607
feature number: 7
train_rmse: 0.004042087916713229
test_rmse: 0.004208997911528394
test_r2: 0.6829602163837639
feature number: 8
train_rmse: 0.004038104419686901
test_rmse: 0.004225565086649968
test_r2: 0.6802614900985136
feature number: 9
train_rmse: 0.004035586388670297
test_rmse: 0.004240850072658861
test_r2: 0.6780859563826654
feature number: 10
train_rmse: 0.004032934594002402
test_rmse: 0.0042578005

train_rmse: 0.004001610842799321
test_rmse: 0.00432198157737647
test_r2: 0.6655902509929797
feature number: 16
train_rmse: 0.003999797349326196
test_rmse: 0.004336293261822109
test_r2: 0.6634053193256173
feature number: 17
train_rmse: 0.003994021834977391
test_rmse: 0.004365128150844366
test_r2: 0.6589991707981507
feature number: 18
train_rmse: 0.003992992345139798
test_rmse: 0.004372736479322428
test_r2: 0.6578906030093357
feature number: 19
train_rmse: 0.003991476242066797
test_rmse: 0.004382908595347933
test_r2: 0.6562879858028038
feature number: 20
train_rmse: 0.003973398584036401
test_rmse: 0.004383622576401361
test_r2: 0.6561726147267152
Test_rmse_min = 1.fea_num: 7  2.rmse: 0.004208997911528394
Test_r2_max = 1. fea_num: 7  2.r2: 0.6829602163837639
Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.004411184114791146
test_rmse: 0.004392649897285058
test_r2: 0.656202133733572
feature number: 3
train_rmse: 0.004383196449609636
test_rmse: 0.004411265060407387
test_r2: 0.65278

train_rmse: 0.004032934594002402
test_rmse: 0.004257800576085822
test_r2: 0.675619091902502
feature number: 11
train_rmse: 0.004029958347752987
test_rmse: 0.004275748147952013
test_r2: 0.6727163074022237
feature number: 12
train_rmse: 0.004027341549779334
test_rmse: 0.004289735883026416
test_r2: 0.670535267547978
feature number: 13
train_rmse: 0.004022968643514835
test_rmse: 0.004313060349615441
test_r2: 0.6666731878584484
feature number: 14
train_rmse: 0.004004516427728336
test_rmse: 0.00430798704235405
test_r2: 0.66773285142819
feature number: 15
train_rmse: 0.004001610842799321
test_rmse: 0.00432198157737647
test_r2: 0.6655902509929797
feature number: 16
train_rmse: 0.003999797349326196
test_rmse: 0.004336293261822109
test_r2: 0.6634053193256173
feature number: 17
train_rmse: 0.003994021834977391
test_rmse: 0.004365128150844366
test_r2: 0.6589991707981507
feature number: 18
train_rmse: 0.003992992345139798
test_rmse: 0.004372736479322428
test_r2: 0.6578906030093357
feature number: 1

# 可以修改第一層的神經元數量

In [14]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(8, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [11]:
fea_num=[4,5,6,7,8] # 要測試的feature數量 神經元10

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_3/Input_C_057.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.006537878092805248
[0.005250384982900598, 0.013070473523084427, 0.00532184645676939, 0.004537622039978524, 0.012190072864897778, 0.00418273710035891, 0.005594708282255634, 0.005031277323998786, 0.005010403485608906, 0.005189254868199522]


test_r2_ave: 0.011189702313375915
[0.4376201405023833, -2.4352667079258845, 0.48861194401168895, 0.6507269759034413, -1.8475200075606817, 0.7013518838350303, 0.4178226305931102, 0.5748758709408587, 0.5982129972598076, 0.5254612955740049]


feature number: 5
test_rmse_ave: 0.005717340832480775
[0.004689715835915975, 0.004536418748496015, 0.007188301083990501, 0.0072749412798651155, 0.005223724999684486, 0.004959550970678011, 0.00467027202810745, 0.0046373708869264065, 0.009835157007471168, 0.0041579554836726216]


test_r2_ave: 0.37443036008013
[0.551316067785917, 0.5861870376087612, 0.06700692897247795, 0.10222523959641872, 0.4771046347578496, 0.5801217692278877, 0.5943188942446227, 0.6388372883766911, -0.54815014169